In [1]:
# Modules
from qiskit.quantum_info import Pauli, SparsePauliOp
from itertools import combinations
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
import numpy as np
from scipy.optimize import minimize
from functools import reduce


In [5]:
# Les valeurs / exemples
#n = 3  n_qubits = P_i, un seul générateur → P_i qubits
q=4 #bits par générateur 
P_i   = [3,2,4,5]          # granularité (3 qubits)
c_i   = [10,8,30,6 ]      # coût linéaire
D_i   = [30,20,25, 27]        # demande par noeud
D_tot=np.sum(D_i)

lam=100
P_ij={(0,1):3, (1,2): 4, (2,3): 8, (3,4): 2, (4,1): 3, (4,0): 7, (4,2): 4, (0,2): 5, (0,3): 9, (1,3): 2 }



In [19]:
liste = list(range(q))
alpha_m=[0,1,2,4,6] # Pour représenter 360 degrés
couples = [(i, j) for i in range(q+1) for j in range(i, q+1)]


x_vars = [(i, j) for i in range(q) for j in range(i+1, q)]
beta_vars = [(i, s, m) for i in liste for m in range(5) for s in range(m+1, 5)] # beta_sm=beta_ms ?????
#c_vars=[(r,i,s) for i in liste for r in range(int(np.log2(P_i[i]+1))) for s in range (r)]

c_vars = [
    (i, s)
    for i in liste
    for s in range(int(np.ceil(np.log2(P_i[i] + 1)))
)
]
c_vars2=[(i,j,p) for i in liste for j in range(i+1, q) for p in range(int(np.ceil(np.log2(P_ij[(i,j)] + 1))))]
c_vars3= [(i,j,p) for i in liste for j in range(i+1, q) for p in range(int(np.ceil(np.log2(P_ij[(i,j)] + 1))))]

all = x_vars + beta_vars + c_vars + c_vars2+ c_vars3
qubit_index = {f"x_{i}_{j}": idx for idx, (i,j) in enumerate(x_vars)}
offset = len(x_vars)
for k, (i,s,m) in enumerate(beta_vars):
    qubit_index[f"beta_{i}_{s}_{m}"] = offset + k
    
offset = len(x_vars) + len(beta_vars)
for k, (i, s) in enumerate(c_vars):
    qubit_index[f"c_{i}_{s}"] = offset + k
    
offset=len(x_vars) + len(beta_vars)+ len(c_vars)
for k, (i,j,p) in enumerate (c_vars2):
    qubit_index[f"c2_{i}_{j}_{p}"]=offset+k
    
offset=len(x_vars) + len(beta_vars)+ len(c_vars)+ len(c_vars3)
for k, (i,j,p) in enumerate (c_vars3):
    qubit_index[f"c3_{i}_{j}_{p}"]=offset+k


n_qubit = len(all)

print(n_qubit)

92


In [7]:
#Pour coder les index
def idx_x(i, j):
    return qubit_index[f"x_{i}_{j}"]

def idx_beta(i, s, m):
    return qubit_index[f"beta_{i}_{s}_{m}"]

def idx_c(i,r):
    return qubit_index[f"c_{i}_{r}"]

def idx_c2(i,j,p):
    return qubit_index[f"c2_{i}_{j}_{p}"]

def idx_c3(i,j,p):
    return qubit_index[f"c3_{i}_{j}_{p}"]

In [8]:
terms = []
B_ij = {(0,1):10, (1,2): 13, (2,3): 14, (3,4): 12, (4,1): 9, (4,0): 7, (4,2): 15, (0,2): 8, (0,3): 9, (1,3): 7 }

for i in range(q):
    for j in range(i+1, q):
        if (i,j) not in B_ij:
            continue
        Bij = B_ij[(i,j)]
        for m in range(5):  # M: nombre de "modes" alpha
            for s in range(m+1,5):  # S: nombre de bits pour l'encodage binaire
                alpha = alpha_m[m]
                coeff = (alpha / 4.0) * (2**s) * (lam + c_i[i]) * Bij

                x_idx = idx_x(i,j)
                b_i_idx = idx_beta(i,s,m)
                b_j_idx = idx_beta(j,s,m)

                # - Z_x * Z_beta_j
                pauli = ['I'] * n_qubit
                pauli[x_idx] = 'Z'
                pauli[b_j_idx] = 'Z'
                terms.append((''.join(pauli), -coeff))

                # + Z_x * Z_beta_i
                pauli = ['I'] * n_qubit
                pauli[x_idx] = 'Z'
                pauli[b_i_idx] = 'Z'
                terms.append((''.join(pauli), coeff))

                # + Z_beta_j
                pauli = ['I'] * n_qubit
                pauli[b_j_idx] = 'Z'
                terms.append((''.join(pauli), coeff))

                # - Z_beta_i
                pauli = ['I'] * n_qubit
                pauli[b_i_idx] = 'Z'
                terms.append((''.join(pauli), -coeff))


In [9]:
for i in range(q):
    for j in range(i+1, q):
        terms.append(('I'*n_qubit, -P_ij[(i,j)]+ 2 * D_i[i]-P_i[i] ))    


In [10]:
for i in range(q):
    ki=int(np.ceil(np.log2(P_i[i] + 1)))
    for r in range(ki):
            c_idx = idx_c(i, r)  # mapping des qubits c_{r,i}
            coeff = 2**(r-1)

            # 1 * coeff (pour le "1" dans (1 - Z))
            terms.append(('I'*n_qubit, coeff))

            # -Z_c * coeff
            pauli = ['I'] * n_qubit
            pauli[c_idx] = 'Z'
            terms.append((''.join(pauli), -coeff))


In [11]:
for i in range(q):
    
    for j in range(i+1,q):
        kij=int(np.ceil(np.log2(P_ij[(i,j)]+1)))

        for p in range(kij):
            c2_idx = idx_c2(i,j, p)  
            coeff = 2**(p-1)

            # 1 * coeff (pour le "1" dans (1 - Z))
            terms.append(('I'*n_qubit, coeff))

            # -Z_c * coeff
            pauli = ['I'] * n_qubit
            pauli[c2_idx] = 'Z'
            terms.append((''.join(pauli), -coeff))

In [12]:
for i in range(q):
    
    for j in range(i+1,q):
        kij=int(np.ceil(np.log2(P_ij[(i,j)]+1)))

        for p in range(kij):
            c3_idx = idx_c3(i,j, p)  
            coeff = 2**(p-1)

            terms.append(('I'*n_qubit, coeff))

            pauli = ['I'] * n_qubit
            pauli[c3_idx] = 'Z'
            terms.append((''.join(pauli), -coeff))

In [13]:
H_c = SparsePauliOp.from_list(terms)


In [14]:
#Définition de L'Hamiltonien de mélange
from qiskit.quantum_info import SparsePauliOp

H_M2 = SparsePauliOp.from_list([
    ('I'*i + 'X' + 'I'*(n_qubit-i-1), 1.0)
    for i in range(n_qubit)
])

print(H_M2)


SparsePauliOp(['XIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'IXIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'IIXIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'IIIXIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'IIIIXIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'IIIIIXIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'IIIIIIXIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'IIIIIIIXIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'IIIIIIIIXIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'IIIIIIIIIXIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIXIIIIIIIIIIIII

In [15]:
from qiskit.circuit.library import PauliEvolutionGate
def U_C(gamma, H):
    gate = PauliEvolutionGate(H, time=gamma)
    return gate.definition  

def U_M(beta, H):
    gate = PauliEvolutionGate(H, time=beta)
    return gate.definition


In [16]:
from qiskit import QuantumCircuit

# Construction du circuit QAOA --> SANS mesure
def build_qaoa_circuit(n, p, gammas, betas, cost_h, mixer_h):
    qc = QuantumCircuit(n)  
    qc.h(range(n))          
    
    for layer in range(p):
        qc.append(U_C(float(gammas[layer]), cost_h), range(n))
        qc.append(U_M(float(betas[layer]), mixer_h), range(n))
    
    return qc 


In [17]:
import pylatexenc
p=2
gammas=[np.pi/4, np.pi/2]
betas=[np.pi/8, np.pi/4]
params=[np.pi/4, np.pi/2, np.pi/8, np.pi/4 ]
cost_h=H_c
mixer_h=H_M2
qc2=build_qaoa_circuit(n_qubit,p,gammas,betas,cost_h,mixer_h)
qc2.measure_all()
qc2.draw("mpl")


# Affichage du résultat très long --> Plusieurs valeurs

In [18]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer, AerSimulator  
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt


qc_transpiled2 = transpile(qc2, AerSimulator())

job = AerSimulator().run(qc_transpiled2, shots=1024)
result = job.result()
counts = result.get_counts()

plot_histogram(counts)


CircuitTooWideForTarget: 'Number of qubits (92) in circuit-171 is greater than maximum (29) in the coupling_map'

In [ ]:
from qiskit import transpile
from qiskit_aer import Aer
import numpy as np
from scipy.optimize import minimize

backend = Aer.get_backend('aer_simulator')

def qaoa_expectation(params):
    p = 2
    gammas = params[:p]
    betas = params[p:]
    
    qc2 = build_qaoa_circuit(n_qubit, p, gammas, betas, H_c, H_M2)
    qc2.save_statevector()
    
    qc_transpiled = transpile(qc2, backend)
    result = backend.run(qc_transpiled).result()
    
    sv = result.data(0)['statevector']
    expval = sv.expectation_value(H_c)
    return -np.real(expval)  # on maximise le coût en minimisant l'opposé




In [ ]:

result = minimize(qaoa_expectation, params, method='COBYLA')

print("Paramètres optimaux :", result.x)
print("Espérance d'énergie minimale :", -result.fun)
